In [1]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import os
import subprocess
import shutil

import librosa
import soundfile as sf
import torch

from pyannote.audio import Pipeline


c:\Users\User\anaconda3\envs\tts_asr\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\anaconda3\envs\tts_asr\lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [3]:
def download_youtube_audio(youtube_id, out_dir="yt_audio"):
    os.makedirs(out_dir, exist_ok=True)

    url = f"https://www.youtube.com/watch?v={youtube_id}"

    cmd = [
        "yt-dlp",
        "-f", "bestaudio/best",
        "--extract-audio",
        "--audio-format", "wav",
        "--audio-quality", "0",
        "--ffmpeg-location", "C:\\ffmpeg\\bin",  # 🔧 แก้ให้ตรงเครื่อง
        "--postprocessor-args", "ffmpeg:-ac 1",  # mono only
        "-o", f"{out_dir}/{youtube_id}.%(ext)s",
        url
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(result.stderr)
        raise RuntimeError("yt-dlp failed")

    print(f"✅ downloaded {youtube_id}")


In [ ]:
os.makedirs("inference_audio", exist_ok=True)

yt_id = "Mrax5_z07Y0" # ดาวหางฮัลเลย์
download_youtube_audio(yt_id, "inference_audio")

✅ downloaded Mrax5_z07Y0


In [ ]:
yt_id = "Gy-MZjiFv2M" # ดอกกระเจียวบาน
download_youtube_audio(yt_id, "inference_audio")

✅ downloaded Gy-MZjiFv2M


In [13]:
# !pip install demucs

In [14]:
import subprocess

def get_vocal_voice(file_path):
    
    subprocess.run([
        "demucs",
        "--two-stems=vocals",
        file_path
    ])

In [15]:
file_path_list = ["inference_audio/Gy-MZjiFv2M.wav", "inference_audio/Mrax5_z07Y0.wav"]

In [16]:
for file_path in file_path_list:
    get_vocal_voice(file_path)

In [17]:
import os
import pandas as pd

import torch
import whisper
from transformers import pipeline

from transformers import logging
logging.set_verbosity_error()

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32


whisper_model = whisper.load_model("large")
whisper_model = whisper_model.to(device)

def whisper_trans(audio_path):
    """Transcribe with OpenAI Whisper (single file)"""
    with torch.no_grad():
        result = whisper_model.transcribe(audio_path)
    return result["text"]

In [19]:
task = "transcribe"

pipe_pathumma = pipeline(
    task="automatic-speech-recognition",
    model="nectec/Pathumma-whisper-th-large-v3",
    torch_dtype=torch_dtype,
    device=device,
)

pipe_pathumma.model.config.forced_decoder_ids = (
    pipe_pathumma.tokenizer.get_decoder_prompt_ids(
        language=None,
        task=task
    )
)

def pathumma_trans(audio_path):
    """Transcribe with Pathumma Whisper (single file)"""
    with torch.no_grad():
        result = pipe_pathumma(audio_path)
    return result["text"]


In [20]:
file_path_list = ["separated/htdemucs/Gy-MZjiFv2M/vocals.wav", "separated/htdemucs/Mrax5_z07Y0/vocals.wav"]

rows = []

for file_path in file_path_list:
    whisper_text = whisper_trans(file_path)
    pathumma_text = pathumma_trans(file_path)

    speaker = file_path.split("/")[0]

    row_dict = {
        "file_path": file_path,
        "whisper_text": whisper_text,
        "pathumma_text": pathumma_text,
        "speaker": speaker
    }

    rows.append(row_dict)

    print(f"File: {file_path} - Whisper: {whisper_text} - Pathumma: {pathumma_text}")

File: separated/htdemucs/Gy-MZjiFv2M/vocals.wav - Whisper: สงสาร Sew goes furtherได้มาพ้อ หนอมแก้มอง ต้องนположวันสงกรานเต้นอยู่หน้าอาหาร เมื่อคืนเทศกาลวันไหลเจ้ายิ้มเข้าวะยอก บอกว่า ฮักไอ้เบิดใจเข้ามาสบที่ตรงไล จะว่า วะใจเหาะตรงกันอยู่กันสองคืน เจ้ามาคอกละไปเห็ดงานก่อนกระเจียวบาน บ่อต้องย่านสี่กล่ำมหาไอ้ก็ทาจมเศร้า กอดรูปเจ้าทั้งน้ำตาเจ้าคงลืมสัญญา สามเดือนกว่าๆ แล้วไปโยสัยอยากเจอ ได้แค่เพียงคิดทอดรอวันได้กอดเธอเฝ้าแต่ฝันและเมื่อกับคำว่า ฮักที่เธอนั้นบอกไอ้บอกหัวจริงหรือยอบ ยานคนสวยนั้นสิ่งมาหลอกให้ไอ้ชำดอกกระเจียวบาน งีดไม่นานก็คล่องสี่เช้าไอ้ก็รอเจ้าอยู่เค้าเป็นผู้สาวผู้ใดหนอนางพource Lady โน้นหนั้นค้าทอดบอกไซซ sac vowได้นอนจนฝา สางยันความstad server มาตะดแม่จนลืมทุกอย่างของก็ส่องเขาโดกกระเจียวบาน ผ่านนาแหล่งเจ้าไปอยู่ใสเค้านาฝนก็โดน Julyน้ำตาของไอ้��서ไ המ� irgendwasใช้ídouhanทอตรึนบทเง�도 Marinจนดอกกระเจียวของไอ้ jamเช่าเจ้ายังบอกเคยมาอยากเจอ ได้แค่เพียงคิดโทษ รอวันได้กอดเธอเฝ้าแต่ฝันและเมื่อกับคำว่าหากที่เธอนั้นบอกอาจจะไม่รู้ แต่คิดว่าเธอจะไม่รู้บอกว่าจริงลื้อหยอบ เงินขุนสวยนั้นซี่

In [21]:
df = pd.DataFrame(rows)
df.to_csv("transcription_inference.csv", index=False)

In [22]:
df.head()

,file_path,whisper_text,pathumma_text,speaker
0,separated/htdemucs/Gy-MZjiFv2M/vocals.wav,สงสาร Sew goes furtherได้มาพ้อ หนอมแก้มอง ต้อง...,ได้มาพ่อน้องแก้มอ้องต้องในวันสงกรานต์ เต้นอยู่...,separated
1,separated/htdemucs/Mrax5_z07Y0/vocals.wav,Thank you. Thank you. Thank you. Thank you. T...,เมื่อเมื่อ มีการปฏิบัติการการการเงินที่สำคัญใน...,separated
